In [1]:
# Import section
import cv2
import numpy as np
from os import scandir, getcwd, rename
import requests
import time
import json
import re
import pandas as pd

In [6]:
a = [1,2,3,4,5,6,7,8,9,10]

In [8]:
a[0:6]

[1, 2, 3, 4, 5, 6]

In [9]:
a[6:12]

[7, 8, 9, 10]

In [2]:
# Getting arrays
ruta = 'C:\\git\\cuponesWong\\CuponesWong\\notebooks_flow\\areas_result\\'

m_widths = np.genfromtxt(ruta+'widths.txt')
m_heights = np.genfromtxt(ruta+'heights.txt')
m_cxs = np.genfromtxt(ruta+'cxs.txt')
m_cys = np.genfromtxt(ruta+'cys.txt')
m_files = np.load(ruta+'paths.npy')
m_azure_flags = np.load(ruta+'azure_flags.npy')

In [3]:
m_files.size, m_azure_flags.size

(12, 12)

In [4]:
# URL variables
vision_base_url = "https://eastus2.api.cognitive.microsoft.com/vision/v2.0/"
# vision_base_url = "https://southcentralus.api.cognitive.microsoft.com/vision/v2.0/"
ocr_url = vision_base_url + "recognizeText?mode=Handwritten"
subscription_key = "db6ddba1087844a58b0329c27b44ff5e"
# subscription_key = "e3d29d8cac4748328976033879af3216"

files = []
jsons = []

i = 1
files_count = 0

# Recorriendo la lista de archivos en la ruta
for i in range(m_files.size):
    # file = m_files[i]
    
    # if files_count>=19:
        # print('Sleeping 65 secs')
        # time.sleep(65)
        # files_count=0
    
    # ruta por archivo
    print("Reading {}".format(m_files[i]),end=" ")
    image = open(m_files[i],"rb")
    d = image.read()
    headers = {'Content-Type': 'application/octet-stream', 'Ocp-Apim-Subscription-Key': subscription_key}

    response = requests.post(ocr_url, headers=headers, data=d)
    
    # Holds the URI used to retrieve the recognized text.
    operation_url = response.headers["Operation-Location"]

    # The recognized text isn't immediately available, so poll to wait for completion.
    analysis = {}
    poll = True
    
    while (poll):
        response_final = requests.get(
            operation_url, headers=headers)
        analysis = response_final.json()
        # print(analysis)
        time.sleep(5)
        if ('recognitionResult' in analysis):
            poll= False 
        if ("status" in analysis and analysis['status'] == 'Failed'):
            poll= False
    
    obj = json.loads(response_final.text)
    # dic[file] = obj
    
    files.append(m_files[i])
    jsons.append(obj)
    image.close()
    files_count +=1
print("Fin consulta Azure : D")

Reading C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164448_00043.jpg Reading C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164503_00537.jpg Reading C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164503_00546.jpg Reading C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164511_00837.jpg Reading C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_EL_B2_20190704114117_00146.jpg Reading C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_EL_B3_20190705154148_00762.jpg Reading C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_EL_B3_20190705154149_00798.jpg Reading C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_LV_B2_20190704151228_00879.jpg Reading C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_MD_B2_20190704102517_00943.jpg Reading C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_MD_B5_2

In [6]:
len(files), len(jsons)

(10, 10)

In [7]:
azure_files = np.array(files)
azure_jsons = np.array(jsons

SyntaxError: unexpected EOF while parsing (<ipython-input-7-ae8b221af76a>, line 2)

In [10]:
for i in range(azure_jsons.size):
# for i in range(0,1):qqqqqqq
    # Getting json info
    j=0
    # Reading the image
    print('Para: {}'.format(azure_files[i]))
    image = cv2.imread(azure_files[i])
    
    for line in azure_jsons[i]['recognitionResult']['lines']:
        # final_str += line['text']
        j+=1
        print('Line {}: {},{}'.format(j,line['text'],line['boundingBox']))
        cv2.rectangle(image,(line['boundingBox'][0],line['boundingBox'][1]),(line['boundingBox'][2],line['boundingBox'][5]),(0,0,0),1)
        cv2.circle(image,center_of(line['boundingBox']), 4, (0,255,255), -1)
    cv2.imshow('image',image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Para: C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_0202072019164448_00043.jpg
Line 1: Nombres y Apellidos:,[31, 34, 307, 36, 306, 72, 30, 71]
Line 2: Blaudi Idali,[313, 2, 718, 13, 716, 79, 311, 69]
Line 3: Raja Revilla.,[42, 69, 465, 70, 464, 137, 42, 137]
Line 4: DNI:,[34, 156, 88, 156, 87, 184, 33, 184]
Line 5: 42650 934 Telefono: 97 7 +84672,[90, 130, 825, 125, 825, 186, 91, 191]
Line 6: Direccion: J. Beamin Quiroga,[30, 199, 817, 193, 818, 242, 31, 247]
Line 7: 569,[41, 255, 155, 250, 157, 293, 40, 299]
Line 8: German,[444, 260, 690, 268, 688, 301, 442, 292]
Line 9: Distrito:,[33, 326, 135, 328, 134, 358, 32, 356]
Line 10: San Martin de porres,[139, 313, 789, 302, 790, 354, 140, 366]
Line 11: E-mail:,[32, 383, 121, 384, 120, 413, 32, 413]
Line 12: maicusi@ hotmail . com,[136, 362, 808, 359, 809, 415, 137, 418]
Line 13: Centro Comercial:,[32, 441, 261, 440, 262, 471, 33, 473]
Line 14: Plaza Norte.,[276, 413, 675, 410, 676, 473, 277, 475]
Para: C:\git\cuponesWong

Para: C:\git\cuponesWong\CuponesWong\notebooks_flow\input\MarzoCompras_MD_B5_20190709165132_00965.jpg
Line 1: Nombres y Apellidos:,[33, 48, 303, 50, 302, 84, 32, 83]
Line 2: Ana Libia,[318, 17, 672, 21, 671, 92, 317, 87]
Line 3: Espinoza Trujillo,[65, 86, 578, 84, 579, 137, 66, 138]
Line 4: DNI:,[34, 168, 88, 169, 87, 196, 33, 197]
Line 5: 70605766,[102, 143, 388, 148, 387, 198, 101, 193]
Line 6: Telefono:,[422, 166, 543, 167, 542, 199, 421, 197]
Line 7: 935 05 43 02,[547, 147, 817, 148, 816, 195, 546, 194]
Line 8: Direccion:,[31, 225, 158, 224, 159, 255, 32, 256]
Line 9: M2. Sy Lt. 3 Parcela 3A,[169, 203, 724, 205, 723, 257, 169, 256]
Line 10: Los Portales de Pachacamac,[61, 261, 767, 257, 768, 307, 62, 312]
Line 11: Distrito:,[32, 340, 133, 341, 133, 371, 31, 370]
Line 12: Villa el Salvador,[164, 316, 611, 319, 610, 366, 163, 363]
Line 13: E-mail:,[32, 396, 120, 396, 120, 426, 32, 426]
Line 14: alet. 2507@gmail.com,[150, 376, 679, 377, 678, 423, 149, 423]
Line 15: Centro Comercial:,[

In [11]:
traslation_dict = {
    'dni':{'tras_x':27,'tras_y':-44,'width':340,'height':75},
    'tel':{'tras_x':27+455,'tras_y':-44,'width':290,'height':75},
    'mail':{'tras_x':60,'tras_y':185,'width':800,'height':65},
    'nomb_ape1':{'tras_x':240,'tras_y':-180,'width':540,'height':80},
    'nomb_ape2':{'tras_x':-40,'tras_y':-105,'width':820,'height':65},
    'dir1':{'tras_x':100,'tras_y':20,'width':670,'height':65},
    'dir2':{'tras_x':-40,'tras_y':72,'width':820,'height':65},
    'distrito':{'tras_x':70,'tras_y':127,'width':800,'height':65},
    'cc':{'tras_x':200,'tras_y':240,'width':800,'height':65}
}

eqs = {
    'Normal':90.00,
    'Low':85.00
}

banned_field_dict = ['DNI:','DNI','DN']+['Telefono:','Telefono']+['E-mail:']+['Nombres y Apellidos:']+['Direccion:']+['Distrito:']+['Centro Comercial:']

lines_dict = {
    'dni_tel':{'tras_x':-40,'tras_y':-45,'width':820,'height':80}
}

n_eqs = {'%':'90',')':'',',':'','$':'5','f':'7','&':'6','+':'7','g':'9','l':'1','o':'0','y':'4','/':'1','.':'','-':''}


In [235]:
# SHOWING IMAGES
y_min = 120
y_max = 213
x_min = 0
x_max = 833

for i in range(2):
    # Grabbing the file
    filename = m_files[i]
    name = filename.split('\\')
    name = name[len(name)-1]
    name = name.split('.')[0]
    
    c_x = int(m_cxs[i])
    c_y = int(m_cys[i])
    
    wi = m_widths[i]
    hi = m_heights[i]
    
    x_minimo = int(c_x-(wi/2))
    x_maximo = int(c_x+(wi/2))
    y_minimo = int(c_y-(hi/2))
    y_maximo = int(c_y+(hi/2))
    
    image = cv2.imread(filename)
    found = False
    
    if (c_x==9999 or c_y==9999):
        cv2.putText(image,'{}'.format('Area no detectada'), (25, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        found = False
    else:
        found = True
        print('{} . {}'.format(i, name))
        
        cupon_areas = dni_rect_from_ce(c_x, c_y+y_min, cv2, image, traslation_dict, lines_dict)
        # print('{}'.format(cupon_areas))
        fields, scores = get_fields_from_json(azure_jsons[i], cupon_areas)
        
        # Parsing numeric fields
        dni, sdni = post_num_field(fields['dni'],scores['dni'],banned_field_dict,n_eqs)
        tel, stel = post_num_field(fields['tel'],scores['tel'],banned_field_dict,n_eqs)
        # Parsing mail
        mail, smail = post_num_field(fields['mail'],scores['mail'],banned_field_dict)
        # Parsing other fields
        nomb1, snomb1 = post_num_field(fields['nomb_ape1'],scores['nomb_ape1'],banned_field_dict,separator=" ")
        nomb2, snomb2 = post_num_field(fields['nomb_ape2'],scores['nomb_ape2'],banned_field_dict,separator=" ")
        dire1, sdire1 = post_num_field(fields['dir1'],scores['dir1'],banned_field_dict,separator=" ")
        dire2, sdire2 = post_num_field(fields['dir2'],scores['dir2'],banned_field_dict,separator=" ")
        dist, sdist = post_num_field(fields['distrito'],scores['distrito'],banned_field_dict,separator=" ")
        cc, scc = post_num_field(fields['cc'],scores['cc'],banned_field_dict,separator=" ")
        # Concatenating two lines fields
        nomb_ape, snomb_ape = nomb1+" "+nomb2, (snomb1+snomb2)/2
        direccion, sdireccion = dire1+" "+dire2, (sdire1+sdire2)/2
        
        # Last trying to get DNI
        if sdni==0:
            # print('{}'.format(post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")))
            st, sc = post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")
            dni = get_dni_from_line(st, n_eqs)
            sdni = 0.0 if dni=="" else sc
        # Last trying to get Tel
        if stel==0:
            # print('{}'.format(post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")))
            st, sc = post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")
            tel = get_tel_from_line(st, n_eqs)
            stel = 0.0 if tel=="" else sc
        
        # Showing fields
        print('DNI {} {}'.format(dni, sdni))
        print('TEL {} {}'.format(tel, stel))
        print('MAIL {} {}'.format(mail, smail))
        print('NOMB {} {}'.format(nomb_ape, snomb_ape))
        # print('NOMB2 {} {}'.format(nomb2, snomb2))
        print('DIR {} {}'.format(direccion, sdireccion))
        # print('DIR2 {} {}'.format(dire2, sdire2))
        print('DIST {} {}'.format(dist, sdist))
        print('CC {} {}'.format(cc, scc))

    cv2.imshow('image',image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

0 . MarzoCompras-0303072019121014_00343
DNI 0740141 87.14285714285714
TEL 536765 87.14285714285714
MAIL  0.0
NOMB Nombres y Apellidos: TERESA HURTADO CASTILLO 86.5
DIR COMA AU . 22 DE AGOSTO) 265 88.75
DIST Tere hurtado of inthe 85.0
CC - COMPS 87.5
1 . MarzoCompras-0303072019121017_00450
DNI DN09900151 85.0
TEL 916881398 85.0
MAIL ims-0663@yahoo,es 90.0
NOMB Mariamilla Chauca Palomino 86.25
DIR JR. FELIPE ARIAS 392 86.66666666666666
DIST J. M . P 88.75
CC Plaza Norte 90.0


In [14]:
# PROCESSING
y_min = 120
y_max = 213
x_min = 0
x_max = 833

filenames = []
dnis = []
tels = []
mails = []
nombres = []
dirs = []
distris = []
ccs = []

sco_dnis = []
sco_tels = []
sco_mails = []
sco_nombres = []
sco_dirs = []
sco_distris = []
sco_ccs = []

for i in range(m_files.size):
    # Grabbing the file
    filename = m_files[i]
    name = filename.split('\\')
    name = name[len(name)-1]
    name = name.split('.')[0]
    
    c_x = int(m_cxs[i])
    c_y = int(m_cys[i])
    
    wi = m_widths[i]
    hi = m_heights[i]
    
    x_minimo = int(c_x-(wi/2))
    x_maximo = int(c_x+(wi/2))
    y_minimo = int(c_y-(hi/2))
    y_maximo = int(c_y+(hi/2))
    
    image = cv2.imread(filename)
    filenames.append(name)
    
    if (c_x==9999 or c_y==9999):
        cv2.putText(image,'{}'.format('Area no detectada'), (25, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        dnis.append("")
        tels.append("")
        mails.append("")
        nombres.append("")
        dirs.append("")
        distris.append("")
        ccs.append("")
        
        sco_dnis.append(0.0)
        sco_tels.append(0.0)
        sco_mails.append(0.0)
        sco_nombres.append(0.0)
        sco_dirs.append(0.0)
        sco_distris.append(0.0)
        sco_ccs.append(0.0)
    else:
        print('{} . {}'.format(i, name))
        
        cupon_areas = dni_rect_from_ce(c_x, c_y+y_min, cv2, image, traslation_dict, lines_dict)
        # print('{}'.format(cupon_areas))
        fields, scores = get_fields_from_json(azure_jsons[i], cupon_areas)
        
        # Parsing numeric fields
        dni, sdni = post_num_field(fields['dni'],scores['dni'],banned_field_dict,n_eqs)
        tel, stel = post_num_field(fields['tel'],scores['tel'],banned_field_dict,n_eqs)
        # Parsing mail
        mail, smail = post_num_field(fields['mail'],scores['mail'],banned_field_dict)
        # Parsing other fields
        nomb1, snomb1 = post_num_field(fields['nomb_ape1'],scores['nomb_ape1'],banned_field_dict,separator=" ")
        nomb2, snomb2 = post_num_field(fields['nomb_ape2'],scores['nomb_ape2'],banned_field_dict,separator=" ")
        dire1, sdire1 = post_num_field(fields['dir1'],scores['dir1'],banned_field_dict,separator=" ")
        dire2, sdire2 = post_num_field(fields['dir2'],scores['dir2'],banned_field_dict,separator=" ")
        dist, sdist = post_num_field(fields['distrito'],scores['distrito'],banned_field_dict,separator=" ")
        cc, scc = post_num_field(fields['cc'],scores['cc'],banned_field_dict,separator=" ")
        # Concatenating two lines fields
        nomb_ape, snomb_ape = nomb1+" "+nomb2, (snomb1+snomb2)/2
        direccion, sdireccion = dire1+" "+dire2, (sdire1+sdire2)/2
        
        # Last trying to get DNI
        if sdni==0:
            # print('{}'.format(post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")))
            st, sc = post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")
            dni = get_dni_from_line(st, n_eqs)
            sdni = 0.0 if dni=="" else sc
        # Last trying to get Tel
        if stel==0:
            # print('{}'.format(post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")))
            st, sc = post_num_field(fields['dni_tel'],scores['dni_tel'],{},separator="")
            tel = get_tel_from_line(st, n_eqs)
            stel = 0.0 if tel=="" else sc
        
        # Showing fields
        # print('DNI {} {}'.format(dni, sdni))
        # print('TEL {} {}'.format(tel, stel))
        # print('MAIL {} {}'.format(mail, smail))
        # print('NOMB {} {}'.format(nomb_ape, snomb_ape))
        # print('NOMB2 {} {}'.format(nomb2, snomb2))
        # print('DIR {} {}'.format(direccion, sdireccion))
        # print('DIR2 {} {}'.format(dire2, sdire2))
        # print('DIST {} {}'.format(dist, sdist))
        # print('CC {} {}'.format(cc, scc))
        
        # Appending into arrays
        dnis.append(dni)
        tels.append(tel)
        mails.append(mail)
        nombres.append(nomb_ape)
        dirs.append(direccion)
        distris.append(dist)
        ccs.append(cc)
        
        sco_dnis.append(sdni)
        sco_tels.append(stel)
        sco_mails.append(smail)
        sco_nombres.append(snomb_ape)
        sco_dirs.append(sdireccion)
        sco_distris.append(sdist)
        sco_ccs.append(scc)

print('Termino :D ')

0 . MarzoCompras_0202072019164448_00043
1 . MarzoCompras_0202072019164503_00537
2 . MarzoCompras_0202072019164503_00546
3 . MarzoCompras_0202072019164511_00837
4 . MarzoCompras_EL_B3_20190705154148_00762
5 . MarzoCompras_EL_B3_20190705154149_00798
6 . MarzoCompras_LV_B2_20190704151228_00879
7 . MarzoCompras_MD_B5_20190709165132_00965
8 . MarzoCompras_XM_B4_20190708153320_00008
9 . MarzoCompras_XM_B5_20190710113243_01121
Termino :D 


In [15]:
bd_azure = pd.DataFrame({'NombreArchivo':np.array(filenames)})
bd_azure['DNI'] = np.array(dnis)
bd_azure['AcertividadDNI'] = np.array(sco_dnis)

bd_azure['Telefono'] = np.array(tels)
bd_azure['AcertividadTelefono'] = np.array(sco_tels)

bd_azure['NombreCompleto'] = np.array(nombres)
bd_azure['AcertividadNombreCompleto'] = np.array(sco_nombres)

bd_azure['Direccion'] = np.array(dirs)
bd_azure['AcertividadDireccion'] = np.array(sco_dirs)

bd_azure['Distrito'] = np.array(distris)
bd_azure['AcertividadDistrito'] = np.array(sco_distris)

bd_azure['Correo'] = np.array(mails)
bd_azure['AcertividadCorreo'] = np.array(sco_mails)

bd_azure['idCampania'] = 4
bd_azure['idUsuario'] = 1
bd_azure['idEstado'] = 2

In [16]:
bd_azure.head()

,NombreArchivo,DNI,AcertividadDNI,Telefono,AcertividadTelefono,NombreCompleto,AcertividadNombreCompleto,Direccion,AcertividadDireccion,Distrito,AcertividadDistrito,Correo,AcertividadCorreo,idCampania,idUsuario,idEstado
0,MarzoCompras_0202072019164448_00043,42650934,85.714286,977784672,85.714286,Blaudi Idali Raja Revilla.,86.250000,J. Beamin Quiroga 569 German,85.000000,San Martin de porres,86.25,maicusi@hotmail.com,87.5,4,1,2
1,MarzoCompras_0202072019164503_00537,40557762,90.000000,927136744,85.000000,,0.000000,&- Sou Juau Bautista # 332 - Palo,86.000000,Son Mortin de tones,86.25,Micasu-800/Hotmail.Com,87.5,4,1,2
2,MarzoCompras_0202072019164503_00546,08642202,87.916667,995382267,87.916667,CAROLINA Cde los MERINO,85.833333,CALLE LOS HOENES # 170 unit. INs. INFANTAS,86.000000,LOS OLIVOS,90.00,CAROLINACHIO10/0AOLMAIL.CO.,85.0,4,1,2
3,MarzoCompras_0202072019164511_00837,,0.000000,98083440,86.666667,JACK cline Min DOa BEcerra,86.250000,Jr. JOSE BAITa. 18 106 Urb . Sam Juanito,86.750000,SAn Juan DE MINAFTons,87.50,soJordyvaGmail.com,89.0,4,1,2
4,MarzoCompras_EL_B3_20190705154148_00762,4714974,85.000000,953065112,86.666667,Mizer SiLUA medina,87.500000,PASAJe: LIBERTAD # 102 ERMITAND,86.666667,Inde Pendencia,87.50,,0.0,4,1,2


In [17]:
# bd_azure.to_csv('azure_result.csv', encoding='utf-8', index = False)

In [9]:
def center_of(bounding_box):
    xs = np.array([bounding_box[0],bounding_box[2],bounding_box[4],bounding_box[6]])
    ys = np.array([bounding_box[1],bounding_box[3],bounding_box[5],bounding_box[7]])
    return int(xs.sum() / xs.size) , int(ys.sum() / ys.size)

def dni_rect_from_ce(x, y, cv2_v, img, areas_dict, lines_dict):
    w = img.shape[1]
    h = img.shape[0]
    
    color = (0, 0, 255)
    thicknes = 2
    
    fields = []
    res_dict = {}
    
    for key in areas_dict.keys():
        tras_vector = np.array([areas_dict[key]['tras_x'], areas_dict[key]['tras_y']])
        point = np.array([x, y]) + tras_vector
        point_2 = point + np.array([areas_dict[key]['width'], areas_dict[key]['height']])
        
        point = np.where(point<=0,0,point)
        point_2[0] = w if point_2[0]>= w else point_2[0]
        point_2[1] = h if point_2[1]>= h else point_2[1]
        
        cv2_v.rectangle(img,(point[0],point[1]),(point_2[0],point_2[1]),color,thicknes)
        res_dict[key] = (point, point_2)
        # print('{} {}'.format(point,point_2))
    
    for key in lines_dict.keys():
        tras_vector = np.array([lines_dict[key]['tras_x'], lines_dict[key]['tras_y']])
        point = np.array([x, y]) + tras_vector
        point_2 = point + np.array([lines_dict[key]['width'], lines_dict[key]['height']])
        
        point = np.where(point<=0,0,point)
        point_2[0] = w if point_2[0]>= w else point_2[0]
        point_2[1] = h if point_2[1]>= h else point_2[1]
        
        # cv2_v.rectangle(img,(point[0],point[1]),(point_2[0],point_2[1]),color,thicknes)
        res_dict[key] = (point, point_2)

    # return res_dict['dni'][0], res_dict['dni'][1], res_dict['tel'][0], res_dict['tel'][1]
    return res_dict

def is_in_box(center, top_left_point, bottom_right_point):
    x = center[0] > top_left_point[0] and center[0] < bottom_right_point[0]
    y = center[1] > top_left_point[1] and center[1] < bottom_right_point[1]
    return (x and y)

def get_fields_from_json(azure_json, areas_dict):
    fields = {}
    scores = {}
    j=0
    
    # Setting empty arrays in result
    for key in areas_dict.keys():
        fields[key] = []
        scores[key] = []

    for line in azure_json['recognitionResult']['lines']:
        j+=1
        # print('Line {}: {},{}'.format(j,line['text'],line['boundingBox']))
        
        # Drawing areas
        cv2.rectangle(image,(line['boundingBox'][0],line['boundingBox'][1]),(line['boundingBox'][2],line['boundingBox'][5]),(0,0,0),1)
        cv2.circle(image,center_of(line['boundingBox']), 4, (0,255,255), -1)
        
        # Searching in areas_dict
        c = center_of(line['boundingBox'])
        t = line['text']
        ws = []
        scos = []
        for w in line['words']:
            ws.append(w['text'])
            try:
                scos.append(w['confidence'])
            except:
                scos.append('Normal')
        
        for key in areas_dict.keys():
            # print('C {} está en {}?'.format(c, areas_dict[key]))
            if is_in_box(c, areas_dict[key][0], areas_dict[key][1]):
                # print('String: {} --> {}'.format(t,key))
                fields[key].append(ws)
                scores[key].append(scos)

    return fields, scores

def basic_digit_clean(string, num_eqs):
    for k in num_eqs.keys():
        string = string.replace(k,num_eqs[k])
    return string

def post_num_field(l_field, l_score, banned, num_eqs={}, separator=""):
    value = ""
    score = 0.0
    ct = 0
    
    for i in range(len(l_field)):
        for j in range(len(l_field[i])):
            # print(j)
            if l_field[i][j] not in banned:
                ct += 1
                value += separator + l_field[i][j]
                # print('{}'.format(l_score[i][j]))
                score += eqs[l_score[i][j]]
    value = basic_digit_clean(value, num_eqs)
    
    return (value.strip(), score/ct) if ct>0 else (value,0.0)

def get_dni_from_line(string, num_eqs):
    a = string.find('DNI:')
    b = string.find('Telefono:')
    if a>=0 and b>=0:
        return basic_digit_clean(string[a+len('DNI:'):b],num_eqs)
    a = string.find('DNI')
    b = string.find('Telefono')
    if a>=0 and b>=0:
        return basic_digit_clean(string[a+len('DNI'):b],num_eqs)
    else:
        return ""

def get_tel_from_line(string, num_eqs):
    b = string.find('Telefono:')
    if b>=0:
        return basic_digit_clean(string[b+len('Telefono:'):],num_eqs)
    b = string.find('Telefono')
    if b>=0:
        return basic_digit_clean(string[b+len('Telefono'):],num_eqs)
    else:
        return ""

In [255]:
# dft = pd.read_csv('azure_result.csv')

In [256]:
# dft.head()

,NombreArchivo,DNI,AcertividadDNI,Telefono,AcertividadTelefono,NombreCompleto,AcertividadNombreCompleto,Direccion,AcertividadDireccion,Distrito,AcertividadDistrito,Correo,AcertividadCorreo,idCampania,idUsuario,idEstado
0,MarzoCompras_0202072019164448_00043,0740141,87.14,536765,87.142857,Nombres y Apellidos: TERESA HURTADO CASTILLO,86.50,COMA AU . 22 DE AGOSTO) 265,88.750000,Tere hurtado of inthe,85.00,NaN,0.0,4,1,1
1,MarzoCompras_0202072019164503_00537,DN09900151,85.00,916881398,85.000000,Mariamilla Chauca Palomino,86.25,JR. FELIPE ARIAS 392,86.666667,J. M . P,88.75,"ims-0663@yahoo,es",90.0,4,1,1


In [20]:
import sqlalchemy as sa
import pyodbc
import time

# dataframe
dataPrueba = bd_azure.fillna('')

# cnxn = pyodbc.connect('Driver={SQL Server}; Server=gyslap316\sql2017; Database=ClienteCupon', autocommit=True)
cnxn = pyodbc.connect('Driver={SQL Server}; Server=192.168.2.55; Database=ClienteCupon; UID=usercupon;PWD=123456789', autocommit=True)
# cnxn = pyodbc.connect('Driver={SQL Server}; Server=gyslap316\sql2017; Database=ClienteCupon', autocommit=True)
crsr = cnxn.cursor()

crsr.fast_executemany = False 

sql = "UPDATE Cupon SET [DNI]=?, [AcertividadDNI]=?, [Telefono]=?, [AcertividadTelefono]=?, [NombreCompleto]=?, [AcertividadNombreCompleto]=?, [Direccion]=?, [AcertividadDireccion]=?, [Distrito]=?, [AcertividadDistrito]=?, [Correo]=?, [AcertividadCorreo]=?, [idCampania]=?, [idUsuario]=?, [idEstado]=? WHERE [NombreArchivo]=?;"
params = [(dataPrueba.at[i,'DNI'],dataPrueba.iloc[i]['AcertividadDNI'],dataPrueba.at[i,'Telefono'],dataPrueba.iloc[i]['AcertividadTelefono'],dataPrueba.iloc[i]['NombreCompleto'],dataPrueba.iloc[i]['AcertividadNombreCompleto'],dataPrueba.iloc[i]['Direccion'],dataPrueba.iloc[i]['AcertividadDireccion'],dataPrueba.iloc[i]['Distrito'],dataPrueba.iloc[i]['AcertividadDistrito'],dataPrueba.iloc[i]['Correo'],dataPrueba.iloc[i]['AcertividadCorreo'],int(dataPrueba.iloc[i]['idCampania']),int(dataPrueba.iloc[i]['idUsuario']),int(dataPrueba.iloc[i]['idEstado']),dataPrueba.at[i,'NombreArchivo']) for i in range(dataPrueba.shape[0])]

t0 = time.time()
crsr.executemany(sql, params)
print(f'{time.time() - t0:.1f} seconds')

0.7 seconds


In [3]:
import numpy as np

In [1]:
ruta = "D:\\GyS\\hans\\bimbot\\testingthreads\\temps\\"

paths_total = []
jsons_total = []
for i in range(4):
    j = np.load(ruta+'n_jsons_{}.npy'.format(i), allow_pickle=True)
    # print('{}'.format(i))

0
1
2
3


In [22]:
j = np.load(ruta+'n_jsons_0.npy', allow_pickle=True)
j2 = np.load(ruta+'n_jsons_1.npy', allow_pickle=True)

In [21]:
j.shape

(2,)

In [23]:
j2.shape

(1,)